In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---
# Drosophila
## Ariss

In [ ]:
ariss = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Ariss/data.h5")
utils.peek(ariss, "build/eye_disc/Ariss")
ariss.obs.head()

In [ ]:
ariss_model = cb.directi.fit_DIRECTi(
    ariss, ariss.uns["seurat_genes"],
    **fixed_model_kwargs
)
ariss.latent = ariss_model.inference(ariss)

In [ ]:
ax = ariss.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/eye_disc/Ariss/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = ariss.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/eye_disc/Ariss/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ariss.write_dataset("build/eye_disc/Ariss/Ariss.h5")

In [ ]:
%%capture capio
ariss_models = [ariss_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    ariss_models.append(cb.directi.fit_DIRECTi(
        ariss, ariss.uns["seurat_genes"],
        **fixed_model_kwargs,
        random_seed=i
    ))
ariss_blast = cb.blast.BLAST(ariss_models, ariss)
ariss_blast.save("build/eye_disc/Ariss")

In [ ]:
with open("build/eye_disc/Ariss/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/eye_disc/Ariss/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(ariss_blast, "build/eye_disc/Ariss")

In [ ]:
%%writefile "build/eye_disc/Ariss/predictable.txt"
cell_ontology_class
cell_type1